# Without Hyperparameter Tuning

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CIFAR-100 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root='./data', train=False, transform=transform)

# Data loaders
batch_size = 100
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layers=2, neurons_per_layer=64, activation='relu'):
        super(NeuralNetwork, self).__init__()
        layers = []
        layers.append(nn.Flatten())
        for _ in range(hidden_layers):
            layers.append(nn.Linear(input_size, neurons_per_layer))
            if activation == 'relu':
                layers.append(nn.ReLU())
            elif activation == 'tanh':
                layers.append(nn.Tanh())
            input_size = neurons_per_layer
        layers.append(nn.Linear(neurons_per_layer, num_classes))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# Model configuration
input_size = 32 * 32 * 3
num_classes = 100
hidden_layers = 2
neurons_per_layer = 64
activation = 'relu'

# Initialize model
model = NeuralNetwork(input_size, num_classes, hidden_layers, neurons_per_layer, activation).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
total_step = len(train_loader)
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Files already downloaded and verified
Epoch [1/5], Step [100/500], Loss: 3.9893
Epoch [1/5], Step [200/500], Loss: 3.8042
Epoch [1/5], Step [300/500], Loss: 3.7301
Epoch [1/5], Step [400/500], Loss: 3.5881
Epoch [1/5], Step [500/500], Loss: 3.4019
Epoch [2/5], Step [100/500], Loss: 3.2439
Epoch [2/5], Step [200/500], Loss: 3.4847
Epoch [2/5], Step [300/500], Loss: 3.2896
Epoch [2/5], Step [400/500], Loss: 3.4442
Epoch [2/5], Step [500/500], Loss: 3.2142
Epoch [3/5], Step [100/500], Loss: 3.4835
Epoch [3/5], Step [200/500], Loss: 3.2540
Epoch [3/5], Step [300/500], Loss: 3.1765
Epoch [3/5], Step [400/500], Loss: 3.2790
Epoch [3/5], Step [500/500], Loss: 3.2251
Epoch [4/5], Step [100/500], Loss: 3.0510
Epoch [4/5], Step [200/500], Loss: 2.8405
Epoch [4/5], Step [300/500], Loss: 3.3211
Epoch [4/5], Step [400/500], Loss: 3.1830
Epoch [4/5], Step [500/500], Loss: 3.7225
Epoch [5/5], Step [100/500], Loss: 3.1632
Epoch [5/5], Step [200/500], Loss: 3.1332
Epoch [5/5], Step [300/500], Loss: 2.9